In [3]:
import pandas as pd
import json
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch(['http://34.95.230.124:9200/'])
import topic


In [4]:
query = {
  "query": {
    "match_all": {}
  },
    "fields": [
    "user.id",
  ],
    "_source": False
}

users_ids = set()
tweets_ids = []

res = helpers.scan(
                client = es,
                query = query, 
                index = 'forest500')

for tweet in res:
    users_ids.add(tweet['fields']['user.id'][0])
    tweets_ids.append(int(tweet['_id']))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [5]:
from twitterapi import TwitterAPI
import tweepy
import threading
import json
import datetime

def parse_date(tweet):
    tweet['created_at'] = datetime.datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    tweet['user']['created_at'] = datetime.datetime.strptime(tweet['user']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    if ('retweeted_status' in tweet):
        tweet['retweeted_status']['created_at'] = datetime.datetime.strptime(tweet['retweeted_status']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
        tweet['retweeted_status']['user']['created_at'] = datetime.datetime.strptime(tweet['retweeted_status']['user']['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
    return tweet

inicialid = max(tweets_ids)
inicialdate = '2000-01-01'

class TwitterTimeLine(TwitterAPI):
    def __init__(self):
        super().__init__()
        self.data = {}

    def _getTweets(self,userids,api,inicialid):
        for i,id in enumerate(userids):
            self.data[id] = []
            print(id)
            
            try:
                for x in tweepy.Cursor(api.user_timeline, id=id,since_id=inicialid,tweet_mode="extended",count=200).items():
    #                    self.data[id].append(x._json)
                    tweet = parse_date(x._json)
                    res = es.index(index="forest500", id=tweet['id'], body=tweet)
            except:
                print('except')
                continue

            # with open('../data/if/{}.json'.format(id), 'w',encoding='utf-8') as json_file:
            #     json.dump(self.data[id], json_file)
            # self.data.pop(id,None)
            es.indices.refresh(index="forest500")
    def getTweets(self, userids,inicialid):
        splitedList = self.splitList(userids,self.keysLen)        
        threads = [threading.Thread(target=self._getTweets, args=(splitedList[i],self.apis[i],inicialid)) for i in range(self.keysLen)]
        print(threads)
        [thread.start() for thread in threads]        
        [thread.join() for thread in threads]

timeLine = TwitterTimeLine()
excp = []
timeLine.getTweets(list(users_ids),inicialid)
excp = pd.DataFrame(excp)
excp.to_csv('../data/if/excepts.csv',index=False,header=False)

[<Thread(Thread-4, initial)>, <Thread(Thread-5, initial)>, <Thread(Thread-6, initial)>, <Thread(Thread-7, initial)>, <Thread(Thread-8, initial)>, <Thread(Thread-9, initial)>, <Thread(Thread-10, initial)>, <Thread(Thread-11, initial)>, <Thread(Thread-12, initial)>, <Thread(Thread-13, initial)>]
963226534793236480
21778607
12989268380685646

126118683
91374604
350862570
71833002
308506248
32171828
419298953
17640219
236941117
246196403
20457806
760031609177464833
202737470
980628876245127168
290081566
89214789
142294033
845236567
253167239
3288805558
4270279454
400343388
79320096
841699231039664128
58354465
80408759
66344591
991868856489664512
14791918
2898375715
927895728722149377
1156866230
1374361930656268292
1235703799584563201
67427568
859167068
2162938044
234423723
1864359570
467368287
589600174
164365554
1470722864
249548988
880760732048031745
120159034
441129326
1387433628
111381319
5838072
911047687679516672
70915740
358087033
135101478
95310018
287788288
136233342
9653018347547

In [2]:
for k in topic.topic:
    
  query = {
  "track_scores": True,
    "query": {
      "query_string": {
        "default_field" : "full_text",
        "query": topic.topic[k]
      },
    },
      "fields": [
      "topic",
    ],
      "_source": False
  }

  res = helpers.scan(
                  client = es,
                  query = query, 
                  index = 'forest500')
  i = 0
  for tweet in res:
      i+=1
      if('fields' in tweet):
        top = tweet['fields']['topic']
      else:
        top = []
      top.append(k)

      update_query = {
        'doc':{
          'topic': top
        }
      }
      
      # update_query = {
      #   'doc':{
      #     'topics': ['default']
      #   }
      # }
      es.update(index='forest500', id=tweet['_id'], body=update_query)


  print(f'{k}: {i}')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


deforestation: 2374
certification: 3911
monitoring: 47314
nature: 16213
climate: 16956
risks: 14309
sourcing: 16141
protocol: 6588
landuse: 18556
commodities: 1292
palm: 2139
paper: 13109
beef: 3890
soy: 2111
social: 3667
esg: 20281
agenda: 2481


In [ ]:
import topic

In [ ]:
es.indices.refresh(index="forest500")